In [32]:
import numpy as np
import cv2
import itertools
import time
from numba import njit,jit,prange,vectorize,int32,float64,int64,cuda
import random
import tensorflow as tf
import glob

In [33]:
quesize = 20
k = 0

In [39]:
input_size = 256*3
@njit
def preprocessing(a):
    a = a.copy_to_host()
    px_train = a.reshape(input_size)
    px_train = np.asarray(px_train).astype(np.int32)

    return px_train

In [35]:
class WiSARDrp:                                 
    
    def __init__(self,input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number):
        self.input_size = input_size
        self.no_of_rand_pix_selec = no_of_rand_pix_selec
        self.nodes = nodes
        self.ram_address_count = ram_address_count
        self.dis_number = dis_number 


    def discriminator(self):
        discriminator = []
        accumulated_pos = []
        my_list = list(range(0,self.input_size))
        for i in range(self.dis_number):  #10
            ram = []
            random.shuffle(my_list)
            for j in range((int)((self.nodes))): #98    
                total_pos = []
                positions = []
                positions = my_list[j*no_of_rand_pix_selec:j*no_of_rand_pix_selec+no_of_rand_pix_selec]
                accumulated_pos.append(positions)
                total_pos = np.vstack(positions)
                #print(total_pos)
                table = []
                dictionary = {}
                
                max = len("{0:b}".format(2**len(total_pos))) - 1
                for i in range(2**len(total_pos)):
                    x = (('0' * max) + "{0:b}".format(i))
                    x = x[len(x)-max:]
                    dictionary[x] = 0
                table.append(dictionary)
                #print(table)
                ram.append(table)

            di = []
            for j in range(len(ram)):
                for i in range(len(ram[j])):
                    for key, value in ram[j][i].items():
                        temp = [key,value]
                        di.append(temp)
                        
            discriminator.append(di)
    
        discriminator = np.asarray(discriminator).astype(np.int32)
        accumulated_pos = np.asarray(accumulated_pos).astype(np.int32)
        return discriminator, accumulated_pos


In [36]:
input_size = 256*3
no_of_rand_pix_selec = 2**(2)     ## ** (must) no_of_rand_pix_selec = 2^(n) where n is 0,1,2...
nodes = int(input_size/no_of_rand_pix_selec)    
ram_address_count = 2**(no_of_rand_pix_selec)
dis_number = 100*100

In [37]:
w = WiSARDrp(input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number)
d, acc_pos = w.discriminator()
print(d.shape)

(10000, 3072, 2)


In [40]:
import time

@cuda.jit
def partdef(part,address_of_that_ram,b,sum_of_ram_output):
    start = cuda.grid(1)      # (1) one dimensional thread grid, returns a single value
    stride = cuda.gridsize(1)
    
    

@njit(parallel = True)
def test_with_bleaching(d,pos,x_test,y_test):
    right = 0
    wrong = 0
    images = x_test
    lable = y_test
    b=1
        
    image = images
    actual_lable = lable
            
    ix = actual_lable       
    t_ratina = pos[int((nodes*ix)):(int)((nodes*ix+nodes))]
                
    sum_of_ram_output = 0
    dis = d[ix]
                
    for i in range(int(nodes)):
        part = dis[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
        ratina_for_one_ram = t_ratina[i]
                    
        n = []                                                                
        for pix in ratina_for_one_ram:
            if image[(pix-1)]>=1:
                n.append(1)
            else:
                n.append(0)
                    
        num = 0
        for i in range(no_of_rand_pix_selec):
            num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
                    
        address_of_that_ram = (int)(num)
        
        threads_per_block =32
        blocks_per_grid = 1
        #out_device = cuda.device_array_like(sum_of_ram_output)
        for key in range(len(part)):
            prt = part[key]
            if prt[0] == address_of_that_ram and prt[1]>=b:
                sum_of_ram_output += 1
        #partdef[blocks_per_grid, threads_per_block](part,address_of_that_ram,b,sum_of_ram_output)
        #sum_of_ram_output = out_device.copy_to_host()
            
    if sum_of_ram_output/nodes >= 0.75:
        right += 1
    else:
        wrong += 1
        
    return right,wrong                
                
                

@njit
def IsFoundinStabilityQueue(stabilityqueue1,i,j,pixel):
    stabilityqueue2 = stabilityqueue1
    c = cuda.local.array(shape = (quesize,3), dtype = int32)    
    #np.zeros((quesize,3),dtype=np.int32) 
    b = cuda.local.array(shape = (4,3), dtype = int32) 
    #np.zeros((4,3),dtype=np.int32)
    for queueIndex in range(quesize):
        queuevalue = np.reshape(stabilityqueue2[i,j,queueIndex,:],(3,1))
        if queuevalue[0][0] == pixel[0][0] and  queuevalue[1][0] == pixel[1][0] and  queuevalue[2][0] == pixel[2][0] :
            b = np.roll(stabilityqueue2[i,j,:queueIndex+1,:],3) 
            c[:queueIndex+1] = b
            c[queueIndex+2:] = stabilityqueue2[i,j,queueIndex+2:,:]
            stabilityqueue2[i,j,:,:] = c
            return stabilityqueue2,True
    return stabilityqueue2,False
    
    
    
#@njit
@cuda.jit
def bewis(frame,w,ID,stabilityqueue,out,index,d,pos):
    start = cuda.grid(1)      # (1) one dimensional thread grid, returns a single value
    stride = cuda.gridsize(1)
    for i in range(start,frame.shape[0],stride):
        for j in range(frame.shape[1]):
            r,g,b = frame[i,j,:]
            #create binary endoder for each pixel
            #a = np.zeros((256,3))
            a = cuda.local.array(shape = (256,3), dtype = int32)
            a[0:r,0] = 1  
            a[0:g,1] = 1
            a[0:b,2] = 1
            #check if pixel color is in front of stability queue
            front = np.reshape(stabilityqueue[i,j,0,:],(3,1)) 
            pixel = np.reshape(frame[i,j,:],(3,1))
            pixel_no = i*10+j
            if front[0][0] == pixel[0][0] and  front[1][0] == pixel[1][0] and  front[2][0] == pixel[2][0] :
                w[i,j] += ID[i,j]
                ID[i,j] += 1
            if front[0][0] != pixel[0][0] or  front[1][0] != pixel[1][0] or  front[2][0] != pixel[2][0] :
                #stabilityqueue,found = IsFoundinStabilityQueue(stabilityqueue,i,j,pixel)
                found = 1
                if found:
                    ID[i,j] = 1
                else:
                    stabilityqueue[i,j,:,:] = np.roll(stabilityqueue[i,j,:,:],1)
                    stabilityqueue[i,j,0,:]= np.reshape(pixel,(3,))
                    w[i,j] = 0
                    ID[i,j] = 1

            if w[i,j] > k:
                pixel_train = preprocessing(a)
                B = 50    
                images = pixel_train
                lable = pixel_no   
                image = images
                num = lable
                all_ram_of_selected_discriminator = d[num]
                t_ratina = pos[(int)(nodes*num):(int)(nodes*num+nodes)]
            
                for inode in range((int)(nodes)):
                    part = all_ram_of_selected_discriminator[(ram_address_count*inode):(ram_address_count*inode+ram_address_count)]
                    ratina_for_one_ram = t_ratina[inode]
                          
                    n = []                                                                
                    for ix in range(len(ratina_for_one_ram)):
                        pix = ratina_for_one_ram[ix]
                        if image[(pix-1)]>=1:
                            n.append(1)    
                        else:
                            n.append(0)
                
                    num = 0
                    for ino in range(no_of_rand_pix_selec):
                        num = (n[ino])*(10**((no_of_rand_pix_selec-1)-ino)) + num
            
                    address_of_that_ram = (int)(num)
                    for key in range(ram_address_count):
                        index = part[key]
                        if index[0] == address_of_that_ram and index[1] < B:
                            index[1] += 1
                        else:
                            if index[1] != 0 and index[1] != 0:
                                index[1] -= 1
                
            #reconstruct colors from neurons and write it on BG image
            pixel_test = preprocessing(a)
            #right=1
            #wrong =0
            right,wrong = test_with_bleaching(d,acc_pos,pixel_test,pixel_no)
            if right:
                out[i,j] = 0.
            if wrong:
                out[i,j] = 1.




stabilityqueue = np.zeros((100,100,quesize,3),dtype=np.int32)   #240,256  #480,640 for bird video, 720,1280 for E_IP1231_Day_1
w = np.zeros((100,100),dtype=np.int32)
ID = np.zeros((100,100),dtype=np.int32)
cap = cv2.VideoCapture('day.avi')
out = np.ones((100,100),dtype=np.float64)
index = np.zeros((2,),dtype=np.int32)
n = 0
no_of_frames = 0
start = time.time()

while(True):
    #for file in glob.glob('/home/iss/project/BEWiS/dataset2014/dataset/baseline/highway/input/*jpg'):
    #frame = cv2.imread(file)
    ret, frame = cap.read()
    #Display the resulting frame
      
    width = 100
    height = 100
    dim = (width, height)
 
    # resize image
    resized = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    cv2.imshow('frame',resized)
    bewis(resized,w,ID,stabilityqueue,out,index,d,acc_pos)
    no_of_frames += 1
    
    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.erode(out,kernel,iterations = 1)
    dilation = cv2.dilate(erosion,kernel,iterations = 1)
    cv2.imshow('out',dilation)
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

end = time.time()
time = end - start
    
fps = no_of_frames/time
print(fps)
cap.release()
cv2.destroyAllWindows()      

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<numba.cuda.compiler.DeviceFunctionTemplate object at 0x7ffa88dcca58>) with argument(s) of type(s): (array(int32, 2d, C))
 * parameterized
In definition 0:
    TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'copy_to_host' of type array(int32, 2d, C)

File "<ipython-input-39-16eb8da92389>", line 4:
def preprocessing(a):
    a = a.copy_to_host()
    ^

[1] During: typing of get attribute at <ipython-input-39-16eb8da92389> (4)

File "<ipython-input-39-16eb8da92389>", line 4:
def preprocessing(a):
    a = a.copy_to_host()
    ^

    raised from /home/iss/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/numba/typeinfer.py:861
In definition 1:
    TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'copy_to_host' of type array(int32, 2d, C)

File "<ipython-input-39-16eb8da92389>", line 4:
def preprocessing

In [3]:
import pycuda.tools
import pycuda.autoinit
import numpy
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
import pycuda.cumath
from pycuda.elementwise import ElementwiseKernel

blocks = 64
block_size = 128
nbr_values = blocks * block_size

print ("Using nbr_values ==", nbr_values)

# Number of iterations for the calculations,
# 100 is very quick, 2000000 will take a while
n_iter = 100000
print ("Calculating %d iterations" % (n_iter))

# create two timers so we can speed-test each approach
start = drv.Event()
end = drv.Event()



mod = SourceModule("""
__global__ void gpusin(float *dest, float *a, int n_iter)
{
  const int i = blockDim.x*blockIdx.x + threadIdx.x;
  for(int n = 0; n < n_iter; n++) {
    a[i] = sin(a[i]);
  }
  dest[i] = a[i];
}
""")

gpusin = mod.get_function("gpusin")

# create an array of 1s
a = numpy.ones(nbr_values).astype(numpy.float32)
# create a destination array that will receive the result
dest = numpy.zeros_like(a)

start.record() # start timing
gpusin(drv.Out(dest), drv.In(a), numpy.int32(n_iter), grid=(blocks,1), block=(block_size,1,1))
end.record() # end timing
# calculate the run length
end.synchronize()
secs = start.time_till(end)*1e-3
print ("SourceModule time and first three results:")
print ("%fs, %s" % (secs, str(dest[:3])))

ModuleNotFoundError: No module named 'pycuda'

In [ ]:
#####################
# Elementwise SECTION
# use an ElementwiseKernel with sin in a for loop all in C call from Python
kernel = ElementwiseKernel(
   "float *a, int n_iter",
   "for(int n = 0; n < n_iter; n++) { a[i] = sin(a[i]);}",
   "gpusin")

a = numpy.ones(nbr_values).astype(numpy.float32)
a_gpu = gpuarray.to_gpu(a)
start.record() # start timing
kernel(a_gpu, numpy.int(n_iter))
end.record() # end timing
# calculate the run length
end.synchronize()
secs = start.time_till(end)*1e-3
print "Elementwise time and first three results:"
print "%fs, %s" % (secs, str(a_gpu.get()[:3]))


In [ ]:
####################################
# Elementwise Python looping SECTION
# as Elementwise but the for loop is in Python, not in C
kernel = ElementwiseKernel(
   "float *a",
   "a[i] = sin(a[i]);",
   "gpusin")

a = numpy.ones(nbr_values).astype(numpy.float32)
a_gpu = gpuarray.to_gpu(a)
start.record() # start timing
for i in range(n_iter):
    kernel(a_gpu)
end.record() # end timing
# calculate the run length
end.synchronize()
secs = start.time_till(end)*1e-3
print "Elementwise Python looping time and first three results:"
print "%fs, %s" % (secs, str(a_gpu.get()[:3]))

In [ ]:
##################
# GPUArray SECTION
# The result is copied back to main memory on each iteration, this is a bottleneck

a = numpy.ones(nbr_values).astype(numpy.float32)
a_gpu = gpuarray.to_gpu(a)
start.record() # start timing
for i in range(n_iter):
    a_gpu = pycuda.cumath.sin(a_gpu)
end.record() # end timing
# calculate the run length
end.synchronize()
secs = start.time_till(end)*1e-3
print "GPUArray time and first three results:"
print "%fs, %s" % (secs, str(a_gpu.get()[:3]))

In [ ]:
#############
# CPU SECTION
# use numpy the calculate the result on the CPU for reference

a = numpy.ones(nbr_values).astype(numpy.float32)
start.record() # start timing
start.synchronize()

for i in range(n_iter):
    a = numpy.sin(a)

end.record() # end timing
# calculate the run length
end.synchronize()
secs = start.time_till(end)*1e-3
print "CPU time and first three results:"
print "%fs, %s" % (secs, str(a[:3]))